In [1]:
import pandas as pd
import random

#### q2_1 (type) who are the investment partners of [org]

In [102]:
people = pd.read_csv("triples//person_type.csv")
people.columns = ["name_id", "relation", "type"]
people

org = pd.read_csv("triples//org_type.csv")
org.columns = ["name_id", "relation", "type"]
org

id_type = people.append(org)
types = dict(zip(id_type.name_id.tolist(), id_type.type.to_list()))

In [81]:
q2_1 = "which organizations are the investment partners of [org]"

In [82]:
fr_inv = pd.read_csv("triples//investor_funding_round.csv")
fr_ip = pd.read_csv("triples//funding_round_investment_partner.csv")

In [85]:
#l1 = pd.DataFrame(ip["iname_id"].value_counts())
l1 = fr_inv["iname_id"].value_counts().rename_axis('iname_id').reset_index(name='counts')
print (l1)
all_investor = l1["iname_id"].to_list()
sample2_1 = random.sample(all_investor, 1000)
print(sample2_1[:10])

                              iname_id  counts
0                           SOSV@e832c    1882
1                  MassChallenge@56b55    1115
2      New_Enterprise_Associates@63147    1111
3                Sequoia_Capital@55f91    1012
4                          Accel@7bf36     998
...                                ...     ...
13123          Postmedia_Network@9c27b       1
13124                      SheEO@5789f       1
13125         Fullstack_Ventures@e2ca7       1
13126       Ratcliffe_Foundation@5a2ee       1
13127              Mathias_Eklof@39480       1

[13128 rows x 2 columns]
['Heliconia_Capital_Management@79b52', 'Zetta_Venture_Partners@c1e96', 'Valeo@b6e2c', 'Invivo_Capital_Partners@e6f59', 'XAnge@78a2e', 'Odyssey_Capital@aca67', 'GPB_Capital_Holdings@55c55', 'ScaleUP_Ventures@9e40f', 'Northeast_Venture_Fund_(_NEVF_)@f6f23', 'yet2Ventures@d1b16']


In [86]:
# store answers for q2_1
qa2_1 = dict()
for org in sample2_1:
    q = q2_1.replace("[org]", "[" + org + "]")
    al = "|".join(list(set(fr_inv.loc[fr_inv["iname_id"] == org, "funding_round_uuid"].to_list())))
    al = fr_inv.loc[fr_inv["iname_id"] == org, "funding_round_uuid"].to_list()
    a = []
    for item in al:
        ans = fr_ip.loc[fr_ip["funding_round_uuid"] == item, "pname_id"].to_list()
        a.extend(ans)
    a = "|".join(list(set(a)))
    #print(a)
    qa2_1.update({q:a})

qa2_1 = pd.DataFrame.from_dict(qa2_1, orient = "index")
qa2_1.reset_index(inplace=True)
qa2_1.columns = ["q","a"]

qa2_1.to_csv("qa//2_1.txt", sep='\t', index=False, header = None)
qa2_1.to_csv("qa//2_1.csv", index=False, header = None)
qa2_1.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)
#qa2_1

### q_2_2 (ordinal) what type of event did [org] participate in the most

In [87]:
q2_2 = "what type of event did [org] participate in the most"

In [91]:
# ea = pd.read_csv("triples//appearance_type.csv")
# et = pd.read_csv("triples//event_type.csv")
ea = pd.read_csv("triples//event_appearance_type.csv")
et = pd.read_csv("triples//event_role.csv")
ea.columns = ["name_id", "appearance_type", "event_name"]
def get_type(x):
    try:
        result = types[x]
    except:
        return float("NaN")
    return result

ea["type"] = ea["name_id"].apply(lambda x: get_type(x))

In [92]:
print(ea.head())
print(et.head())

                 name_id appearance_type      event_name          type
0  CDW_Corporation@0ed79         sponsor  Adobe_MAX_2014  organization
1        Microsoft@9ffbb         sponsor  Adobe_MAX_2014  organization
2          Twitter@1f709         sponsor  Adobe_MAX_2014  organization
3               HP@a7038         sponsor  Adobe_MAX_2014  organization
4            Intel@75ee5         sponsor  Adobe_MAX_2014  organization
               name relation event_roles
0       SF_New_Tech  type_of  conference
1    Adobe_MAX_2014  type_of  conference
2  ad:tech_New_York  type_of  conference
3   Engadget_Expand  type_of  conference
4   Engadget_Expand  type_of        expo


In [95]:
ea1 = ea[ea["type"] == "organization"]
l2_2 = ea1["name_id"].value_counts().rename_axis('name_id').reset_index(name='counts')
print (l2_2)

#sample2_2 = l2_2["name_id"].to_list()[:50]

sample2_2 = random.sample(l2_2["name_id"].to_list(), 1000)
print(sample2_2[:10])


                                                 name_id  counts
0                                        Microsoft@9ffbb     777
1                                              IBM@a134c     654
2                                            Intel@75ee5     492
3                                           Google@27711     440
4                                            Cisco@0065a     414
...                                                  ...     ...
76039  Escrow_London_/_Escrow_London_North_America@80d34       1
76040                                 Essentialnet@2aaf3       1
76041                                       Esteem@33242       1
76042                             Ethos_Technology@56980       1
76043                                  Thoma_Bravo@3f35a       1

[76044 rows x 2 columns]
['QuSecure@1fce4', 'MMGuardian@452b1', 'ChungpaEMT@5fa51', 'Systems_and_Electronic_Development_FZCO_(SEDCO)@94326', 'Mobility_Work@ca708', 'Introhive@18efd', 'West_Agile_Labs@f9f3b', 'GET_eHealt

In [96]:
# store answers for q2_1
qa2_2 = dict()
event_types = et.event_roles.unique().tolist()

for org in sample2_2:
    q = q2_2.replace("[org]", "[" + org + "]")
    #al = "|".join(list(set(fr_inv.loc[fr_inv["iname_id"] == org, "funding_round_uuid"].to_list())))
    al = ea.loc[ea["name_id"] == org, "event_name"].to_list()
    a = []
    event_count = dict(zip(event_types, [0]*len(event_types)))

    for item in al:
        ans = et.loc[et["name"] == item, "event_roles"].to_list()
        a.extend(ans)
        
    for item in a:
        event_count[item] += 1
    
    d = {}
    for k, v in event_count.items():
        d.setdefault(v, []).append(k)
   
    answer = "|".join(d[max(d)])
    #print(answer)
    #print(event_count)
    
    qa2_2.update({q:answer})

qa2_2 = pd.DataFrame.from_dict(qa2_2, orient = "index")
qa2_2.reset_index(inplace=True)
qa2_2.columns = ["q","a"]

qa2_2.to_csv("qa//2_2.txt", sep='\t', index=False, header = None)
qa2_2.to_csv("qa//2_2.csv", index=False, header = None)
qa2_2.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)
#qa2_1

### q_2_3 (ordinal) what type of event did [person] participate in the most

In [103]:
q2_3 = "what type of event did [person] participate in the most"

In [104]:
ea2 = ea[ea["type"] == "person"]
l2_3 = ea2["name_id"].value_counts().rename_axis('name_id').reset_index(name='counts')
print (l2_3)

#sample2_3 =  l2_3["name_id"].to_list()[:50]

sample2_3 = random.sample(l2_3["name_id"].to_list(), 1000)
print(sample2_3[:10])

                                      name_id  counts
0       Thomas_Balkizas_MSC,_MEng,_MIoD@99a7d     120
1                         Jason_Barnard@1ef61      92
2                          Chris_Fregly@979d2      56
3                  Ahmadreza_Rofougaran@e5ffb      56
4                         Andrew_Martin@89da0      51
...                                       ...     ...
153342                   Beth_Yurchisin@c5f05       1
153343                    Nadim_Tawileh@65959       1
153344               Philipp_Burckhardt@63282       1
153345                         Dan_York@9c6c2       1
153346               Doris_(Yiyang)_Guo@483fd       1

[153347 rows x 2 columns]
['Jacques_Bonifay@bedf8', 'Joao_Rufino@d7f04', 'Gregory_Pickett@87213', 'Milana_Cap@15934', 'Kevin_Hunt@e10bd', 'Gilbert_Hill@748e4', 'Roland_Gossage@d1c86', 'Thomas_Eisenmann@ca758', 'Fredric_Karén@a3ffa', 'Emanuele_Madeddu@56d56']


In [105]:
# store answers for q2_1
qa2_3 = dict()
event_types = et.event_roles.unique().tolist()

for p in sample2_3:
    q = q2_3.replace("[person]", "[" + p + "]")
    #al = "|".join(list(set(fr_inv.loc[fr_inv["iname_id"] == org, "funding_round_uuid"].to_list())))
    al = ea.loc[ea["name_id"] == p, "event_name"].to_list()
    a = []
    event_count = dict(zip(event_types, [0]*len(event_types)))

    for item in al:
        ans = et.loc[et["name"] == item, "event_roles"].to_list()
        a.extend(ans)
        
    for item in a:
        event_count[item] += 1
    
    d = {}
    for k, v in event_count.items():
        d.setdefault(v, []).append(k)
   
    answer = "|".join(d[max(d)])
    #print(answer)
    #print(event_count)
    
    qa2_3.update({q:answer})

qa2_3 = pd.DataFrame.from_dict(qa2_3, orient = "index")
qa2_3.reset_index(inplace=True)
qa2_3.columns = ["q","a"]

qa2_3.to_csv("qa//2_3.txt", sep='\t', index=False, header = None)
qa2_3.to_csv("qa//2_3.csv", index=False, header = None)
qa2_3.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)
#qa2_1

### q3_1 (type) The organizations that are acquired by [org] are from what category groups

In [119]:
q3_1 = "The organizations that are acquired by [org] are from what category groups"

In [115]:
ar = pd.read_csv("triples//is_acquirer.csv")
ae = pd.read_csv("triples//is_acquiree.csv")
cg = pd.read_csv("triples//category.csv")

In [116]:
print(ar.head())
print(ae.head())
print(cg.head())

                      rname_id     relation  \
0  Fox_Interactive_Media@1b78b  is_acquirer   
1                 Google@27711  is_acquirer   
2      CBS_Entertainment@2e4ea  is_acquirer   
3                 Google@27711  is_acquirer   
4       Scripps_Networks@a4b35  is_acquirer   

                                      uuid  
0  ac@c62807f2-3487-f63b-e7fc-559b752ed44c  
1  ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca  
2  ac@01bd2058-ccf8-6102-f253-9224a67a592b  
3  ac@831becd6-e913-b9f5-ac09-2bff915f9453  
4  ac@9da90ee7-aad7-e0b4-8456-839363032e5e  
               ename_id     relation                                     uuid
0         Flektor@330ba  is_acquiree  ac@c62807f2-3487-f63b-e7fc-559b752ed44c
1    GrandCentral@e1192  is_acquiree  ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca
2         Last.fm@85f2d  is_acquiree  ac@01bd2058-ccf8-6102-f253-9224a67a592b
3  Kosiso_-_store@a9f41  is_acquiree  ac@831becd6-e913-b9f5-ac09-2bff915f9453
4      Recipezaar@985a0  is_acquiree  ac@9da90ee7-aad7-e0

In [117]:
l3_1 = ar["rname_id"].value_counts().rename_axis('rname_id').reset_index(name='uuid')
print (l3_1)

#sample3_1 =  l3_1["name_id"].to_list()[:1000]

sample3_1 = random.sample(l3_1["rname_id"].to_list(), 1000)
print(sample3_1[:10])

                          rname_id  uuid
0                  Gallagher@3050a   405
1          HUB_International@c9b7f   331
2                     Google@27711   248
3                  Microsoft@9ffbb   247
4       CVC_Capital_Partners@a1cd3   238
...                            ...   ...
44445  South_Oakland_Shelter@6a088     1
44446      Backstage_Capital@91d5d     1
44447            Iglu_Cruise@5f11c     1
44448             Novametrix@81a11     1
44449             MarketTime@98bf3     1

[44450 rows x 2 columns]
['Graphene_NanoChem@9f870', 'Tritel_BV@ba7f1', 'The_Turbon_Group@ba095', 'Provide_Commerce@3ad11', 'Capitol_Title@c2a86', 'Coastal_Federal_Credit_Union@ecaaa', 'Schon_Klinik@a571c', 'HardCloud@35e63', 'CJK_Engineering@20114', 'Celonis@228e6']


In [123]:
qa3_1 = dict()

for c in sample3_1:
    q = q3_1.replace("[org]", "[" + c + "]")
    #al = "|".join(list(set(fr_inv.loc[fr_inv["iname_id"] == org, "funding_round_uuid"].to_list())))
    step1 = ar.loc[ar["rname_id"] == c, "uuid"].to_list()
    ans1 = []
    
    for item1 in step1:
        ans1.extend(ae.loc[ae["uuid"] == item1, "ename_id"].to_list())
        
    ans2 = []
    for item2 in ans1:
        ans2.extend(cg.loc[cg["name_id"] == item2, "category_list"].to_list())
    
   
    answer = "|".join(ans2)
    #print(answer)    
    qa3_1.update({q:answer})

qa3_1 = pd.DataFrame.from_dict(qa3_1, orient = "index")
qa3_1.reset_index(inplace=True)
qa3_1 = qa3_1.dropna()
qa3_1.columns = ["q","a"]

qa3_1.to_csv("qa//3_1.txt", sep='\t', index=False, header = None)
qa3_1.to_csv("qa//3_1.csv", index=False, header = None)
qa3_1.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)
#qa2_1

### q3_2 (type + explicit temporal) What are some [event] that took place in [coun] during [year]

In [125]:
q3_2 = "What are some [event] that took place in [coun] during [year]"

In [129]:
ec = pd.read_csv("triples//event_country.csv")
es = pd.read_csv("triples//event_start.csv")
er = pd.read_csv("triples//event_role.csv")

In [137]:
print(ec.head())
es["started_on"] = es["started_on"].apply(lambda x: x[:4])
print(es.head())
print(er.head())

               name      relation country_code
0       SF_New_Tech  country_code          USA
1    Adobe_MAX_2014  country_code          USA
2  ad:tech_New_York  country_code          USA
3   Engadget_Expand  country_code          USA
4        Wired_2014  country_code          GBR
               name    relation started_on
0       SF_New_Tech  started_on       2014
1    Adobe_MAX_2014  started_on       2014
2  ad:tech_New_York  started_on       2014
3   Engadget_Expand  started_on       2014
4        Wired_2014  started_on       2014
               name relation event_roles
0       SF_New_Tech  type_of  conference
1    Adobe_MAX_2014  type_of  conference
2  ad:tech_New_York  type_of  conference
3   Engadget_Expand  type_of  conference
4   Engadget_Expand  type_of        expo


In [178]:
l3_21 = er["event_roles"].value_counts().rename_axis('event_roles').reset_index(name='name')
print (l3_21)

#sample3_2 = random.sample(l3_2["event_roles"].to_list(), 5)
sample3_21 = l3_21["event_roles"].to_list()[:]
print(sample3_21)

    event_roles   name
0    conference  13698
1         event   4082
2        meetup   3555
3    networking   3551
4          expo   1942
5       seminar   1765
6      demo_day   1347
7   competition   1218
8         class    818
9       webinar    806
10      virtual    643
11    hackathon    612
12     festival    542
['conference', 'event', 'meetup', 'networking', 'expo', 'seminar', 'demo_day', 'competition', 'class', 'webinar', 'virtual', 'hackathon', 'festival']


In [180]:
l3_22 = ec["country_code"].value_counts().rename_axis('country_code').reset_index(name='name')
print (l3_22)

#sample3_22 = random.sample(l3_22["country_code"].to_list(), 10)
sample3_22 = l3_22["country_code"].to_list()[:20]
print(sample3_22)

    country_code  name
0            USA  9643
1            GBR  1976
2            DEU   820
3            CAN   547
4            IND   453
..           ...   ...
126          AFG     1
127          TTO     1
128          BWA     1
129          SDN     1
130          CMR     1

[131 rows x 2 columns]
['USA', 'GBR', 'DEU', 'CAN', 'IND', 'FRA', 'SGP', 'ESP', 'AUS', 'NLD', 'ARE', 'CHE', 'CHN', 'HKG', 'ITA', 'BRA', 'JPN', 'ISR', 'RUS', 'IRL']


In [181]:
l3_23 = es["started_on"].value_counts().rename_axis('started_on').reset_index(name='name')
print (l3_23)

sample3_23 = [2015, 2016, 2017, 2018, 2019, 2020, 2021]
print(sample3_23)

   started_on  name
0        2018  4634
1        2017  3725
2        2019  3483
3        2015  2945
4        2016  2857
5        2020  2615
6        2021  1505
7        2014  1031
8        2013   179
9        2022    71
10       2012    54
11       2011    42
12       2010    18
13       2007    15
14       2006    12
15       1999    11
16       1998    11
17       2009    11
18       2008    11
19       1901     9
20       1920     7
21       2001     6
22       2000     5
23       2003     5
24       2004     4
25       2005     4
26       1904     3
27       1997     2
28       1921     2
29       1995     2
30       1988     2
31       2002     2
32       1990     2
33       1918     2
34       1996     1
35       1993     1
36       1994     1
37       1970     1
38       1987     1
39       1914     1
40       1902     1
41       1926     1
42       2023     1
43       1910     1
44       1989     1
[2015, 2016, 2017, 2018, 2019, 2020, 2021]


In [182]:
ec = ec[["name", "country_code"]]
er = er[["name", "event_roles"]]
es = es[["name", "started_on"]]
print(ec.head())
print(es.head())
print(er.head())

               name country_code
0       SF_New_Tech          USA
1    Adobe_MAX_2014          USA
2  ad:tech_New_York          USA
3   Engadget_Expand          USA
4        Wired_2014          GBR
               name started_on
0       SF_New_Tech       2014
1    Adobe_MAX_2014       2014
2  ad:tech_New_York       2014
3   Engadget_Expand       2014
4        Wired_2014       2014
               name event_roles
0       SF_New_Tech  conference
1    Adobe_MAX_2014  conference
2  ad:tech_New_York  conference
3   Engadget_Expand  conference
4   Engadget_Expand        expo


In [183]:
new = pd.merge(er, ec, on = "name")
new = pd.merge(new, es, on = "name")
print(new)

                                                    name  event_roles  \
0                                            SF_New_Tech   conference   
1                                         Adobe_MAX_2014   conference   
2                                       ad:tech_New_York   conference   
3                                        Engadget_Expand   conference   
4                                        Engadget_Expand         expo   
...                                                  ...          ...   
29387                                      PAINWeek_2021   conference   
29388                     TecHome_Builder_Summit_Seattle   conference   
29389  AVCJ_PRIVATE_EQUITY_&_VENTURE_FORUM_(34th_ANNU...   conference   
29390  AVCJ_PRIVATE_EQUITY_&_VENTURE_FORUM_(34th_ANNU...   networking   
29391  Harvard_Business_School_New_Venture_Competitio...  competition   

      country_code started_on  
0              USA       2014  
1              USA       2014  
2              USA       20

In [184]:
qa3_2 = dict()

def select_ans(nes):
    (new["country_code"] == c) and (new["started_on"] == y) and (new["event_roles"]) == r

for r in sample3_21:
    for c in sample3_22:
        for y in sample3_23:
            q = q3_2.replace("[event]", "[" + r + "]")
            q = q.replace("[coun]", "[" + c + "]")
            q = q.replace("[year]", "[" + str(y) + "]")
            
            mask = (new['country_code']==c) & (new['started_on']==str(y)) & (new["event_roles"] == r)
            answerdf = new[mask]
            #print(answerdf)
            if len(answerdf) > 0:
                a = answerdf["name"].tolist()
                #print(q)
                #print(a)
                answer = "|".join(a)
                #print(answer)
                try:
                    qa3_2.update({q:answer})
                except:
                    continue

qa3_2 = pd.DataFrame.from_dict(qa3_2, orient = "index")
qa3_2.reset_index(inplace=True)
qa3_2 = qa3_2.dropna()
qa3_2.columns = ["q","a"]

qa3_2.to_csv("qa//3_2txt", sep='\t', index=False, header = None)
qa3_2.to_csv("qa//3_2.csv", index=False, header = None)
qa3_2.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)
#qa2_1

### q3_3 (ordinal) What are the biggest [category] organizations in [coun]

In [186]:
q3_3 = "What are the biggest [category] organizations in [coun]"

In [187]:
oc = pd.read_csv("triples//org_country.csv")
oe = pd.read_csv("triples//org_employee_count.csv")
print(oc.head())
print(oe.head())

                 name_id      relation country_code
0         Wetpaint@33abd  country_code          USA
1             Zoho@7efc7  country_code          USA
2             Digg@553d0  country_code          USA
3  Omidyar_Network@a8eec  country_code          USA
4         Facebook@04fbb  country_code          USA
                 name_id        relation employee_count
0         Wetpaint@33abd  employee_count         51-100
1             Zoho@7efc7  employee_count      1001-5000
2             Digg@553d0  employee_count         51-100
3  Omidyar_Network@a8eec  employee_count        101-250
4         Facebook@04fbb  employee_count         10000+


In [258]:
l3_31 = oc["country_code"].value_counts().rename_axis('country_code').reset_index(name='counts')
print (l3_31)

#sample3_31 =  l3_31["country_code"].to_list()[50:55]
sample3_31 = random.sample(l3_31["country_code"].to_list(), 15)
print(sample3_31[:10])


l3_32 = cg["category_list"].value_counts().rename_axis('category_list').reset_index(name='counts')
print (l3_32)
sample3_32 =  random.sample(l3_32["category_list"].to_list(), 10)
print(sample3_32)

    country_code  counts
0            USA  644167
1            GBR  111975
2            IND   62839
3            DEU   55360
4            CAN   54831
..           ...     ...
211          COK       1
212          TKM       1
213          SJM       1
214          ERI       1
215          BVT       1

[216 rows x 2 columns]
['BMU', 'SVK', 'MWI', 'SYR', 'AZE', 'AGO', 'PRT', 'UKR', 'BFA', 'JOR']
              category_list  counts
0                  Software  218600
1    Information_Technology  206780
2                Consulting  121849
3             Manufacturing  120584
4        Financial_Services  116207
..                      ...     ...
738                 Darknet       9
739                    Roku       7
740                    RISC       4
741        Ultimate_Frisbee       3
742            Generation_Y       1

[743 rows x 2 columns]
['Corrections_Facilities', 'Network_Hardware', 'Sustainability', 'Consumer_Lending', 'Price_Comparison', 'Web_Browsers', 'Photo_Editing', 'Finance', 

In [259]:
qa3_3= dict()
new2 = pd.merge(oc, cg, on  = "name_id")
print(new2)

                  name_id    relation_x country_code relation_y  \
0          Wetpaint@33abd  country_code          USA   category   
1          Wetpaint@33abd  country_code          USA   category   
2          Wetpaint@33abd  country_code          USA   category   
3              Zoho@7efc7  country_code          USA   category   
4              Zoho@7efc7  country_code          USA   category   
...                   ...           ...          ...        ...   
3999848  LiveBeyond@f675c  country_code          USA   category   
3999849  LiveBeyond@f675c  country_code          USA   category   
3999850  LiveBeyond@f675c  country_code          USA   category   
3999851  LiveBeyond@f675c  country_code          USA   category   
3999852  LiveBeyond@f675c  country_code          USA   category   

                   category_list  
0                     Publishing  
1                   Social_Media  
2        Social_Media_Management  
3                Cloud_Computing  
4                  C

In [260]:
def employee_num(x):
    if x == "10000+":
        return 10000
    elif x == "unknown":
        return 0
    elif "-" in x:
        x = int(x.split("-")[0])
    
    return x

oe["employee_count_num"] = oe["employee_count"].apply(lambda x: employee_num(x))
oe

,name_id,relation,employee_count,employee_count_num
0,Wetpaint@33abd,employee_count,51-100,51
1,Zoho@7efc7,employee_count,1001-5000,1001
2,Digg@553d0,employee_count,51-100,51
3,Omidyar_Network@a8eec,employee_count,101-250,101
4,Facebook@04fbb,employee_count,10000+,10000
...,...,...,...,...
1642136,McGuire_Memorial@90fb3,employee_count,251-500,251
1642137,Success_Chain@ea3ac,employee_count,1-10,1
1642138,Capital_Estate@9a7b8,employee_count,1-10,1
1642139,FABAY@ff9d8,employee_count,1001-5000,1001


In [261]:
progress = 0
qa3_3 = dict()
for coun in sample3_31:
    for ca in sample3_32:
        progress += 1
        print(progress)
        print(coun)
        q = q3_3.replace("[coun]", "[" + coun + "]")
        q = q.replace("[category]", "["+ca+"]")
        
        mask2 = (new2['country_code']==coun) & (new2['category_list']==ca)
        answerdf2 = new2[mask2]
        #print(answerdf)
        if len(answerdf2) > 0:
            step1 = answerdf2["name_id"].tolist()
            step2 = dict()
            for item1 in step1:
                ans1 = oe.loc[oe["name_id"] == item1, "employee_count_num"].to_list()[0]
                step2.update({item1: ans1})
                
            d = {}
            #print(step2)
            if len(step2.keys()) > 0:
                for k, v in step2.items():
                    d.setdefault(v, []).append(k)
               
                answer = "|".join(d[max(d)])
   
                qa3_3.update({q:answer})
                print(q, answer)
        else:
            print("done")
            continue

1
BMU
done
2
BMU
done
3
BMU
done
4
BMU
What are the biggest [Consumer_Lending] organizations in [BMU] Triton_Container_International@17fc7
5
BMU
done
6
BMU
What are the biggest [Web_Browsers] organizations in [BMU] Central_European_Media_Enterprises@3382c
7
BMU
done
8
BMU
What are the biggest [Finance] organizations in [BMU] Brookfield_Business_Partners@f9ae6
9
BMU
done
10
BMU
done
11
SVK
done
12
SVK
What are the biggest [Network_Hardware] organizations in [SVK] Virte@b131f
13
SVK
What are the biggest [Sustainability] organizations in [SVK] Bioenergetická_spoločnosť@5ab4d|Grav@c57c4|Biorafineria@2a8a1|E-Vision@2d93e
14
SVK
done
15
SVK
What are the biggest [Price_Comparison] organizations in [SVK] Pricewise@0b702
16
SVK
done
17
SVK
What are the biggest [Photo_Editing] organizations in [SVK] Zoner@5d04f
18
SVK
What are the biggest [Finance] organizations in [SVK] Tatra_banka@5bd8b
19
SVK
What are the biggest [Laundry_and_Dry-cleaning] organizations in [SVK] Lavant@3284d
20
SVK
What are t

In [262]:
print(qa3_3)
qa3_3 = pd.DataFrame.from_dict(qa3_3, orient = "index")
qa3_3.reset_index(inplace=True)
qa3_3 = qa3_3.dropna()
qa3_3.columns = ["q","a"]

qa3_3.to_csv("qa//3_3.txt", sep='\t', index=False, header = None)
qa3_3.to_csv("qa//3_3.csv", index=False, header = None)
qa3_3.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)

{'What are the biggest [Consumer_Lending] organizations in [BMU]': 'Triton_Container_International@17fc7', 'What are the biggest [Web_Browsers] organizations in [BMU]': 'Central_European_Media_Enterprises@3382c', 'What are the biggest [Finance] organizations in [BMU]': 'Brookfield_Business_Partners@f9ae6', 'What are the biggest [Network_Hardware] organizations in [SVK]': 'Virte@b131f', 'What are the biggest [Sustainability] organizations in [SVK]': 'Bioenergetická_spoločnosť@5ab4d|Grav@c57c4|Biorafineria@2a8a1|E-Vision@2d93e', 'What are the biggest [Price_Comparison] organizations in [SVK]': 'Pricewise@0b702', 'What are the biggest [Photo_Editing] organizations in [SVK]': 'Zoner@5d04f', 'What are the biggest [Finance] organizations in [SVK]': 'Tatra_banka@5bd8b', 'What are the biggest [Laundry_and_Dry-cleaning] organizations in [SVK]': 'Lavant@3284d', 'What are the biggest [Recreational_Vehicles] organizations in [SVK]': 'Mad@c9d1d', 'What are the biggest [Finance] organizations in [MW

#### q2_4 (type) When did [org] release its ipo

In [282]:
q2_4 = "When did [org] release its ipo"

In [283]:
oi = pd.read_csv("triples//org_ipo.csv")
ip = pd.read_csv("triples//ipo_public_on.csv")
ip["went_public_on"] = ip["went_public_on"].apply(lambda x: x[:4])
print(oi.head())
print(ip.head())

                         name_id relation  \
0                Microsoft@9ffbb  org_ipo   
1  The_Walt_Disney_Company@f5695  org_ipo   
2                     DivX@81ba9  org_ipo   
3                 XO_Group@aa14b  org_ipo   
4             BMC_Software@89245  org_ipo   

                                       uuid  
0  ipo@72d30ebd-53ef-2486-6c29-22785c5173ce  
1  ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4  
2  ipo@a265c6f6-4b96-4079-096a-967a37f3da2b  
3  ipo@ee426509-826e-5dd0-9309-e79c8f384904  
4  ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e  
                                       uuid        relation went_public_on
0  ipo@72d30ebd-53ef-2486-6c29-22785c5173ce  went_public_on           1986
1  ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4  went_public_on           1978
2  ipo@a265c6f6-4b96-4079-096a-967a37f3da2b  went_public_on           2006
3  ipo@ee426509-826e-5dd0-9309-e79c8f384904  went_public_on           1999
4  ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e  went_public_on           1988


In [284]:
sample2_4 = random.sample(oi.name_id.to_list(), 1000)
print(sample2_4[:20])

['Groupe_Crit@17548', 'Techcom_Capital_Vietnam@f1be6', 'Termo2Power_SA@59ec5', 'Delo_Prodaja@fbbdc', 'PT_Media_Nusantara_Citra_Tbk._(MNC_Media_Group)@43aea', 'UTStarcom@ff4d4', 'Kornit_Digital@54f43', 'SatCon_Technology_Corporation@821eb', 'JSW_PACIFIC_CORP@b57b2', 'Cathay_Chemical_Works@79428', 'Tatfook@1326d', 'Cogobuy_Group@a0680', 'ARC_Minerals@cc0e2', 'Mitsuba@c7df4', 'Beopan_AD@f4ac2', 'Konark_Synthetic@f0e7f', 'Hertz@84be9', 'Perdana_Gapuraprima@de60c', 'Brigade_Group@25f62', 'Saizhuo_Pharmacy@8a173']


In [285]:
qa2_4 = dict()

for org in sample2_4:
    q = q2_4.replace("[org]", "[" + org + "]")
    step1 = oi.loc[oi["name_id"] == org, "uuid"].to_list()

    step2 = []
    for item in step1:
        ans = ip.loc[ip["uuid"] == item, "went_public_on"].to_list()[0]
        step2.append(ans)
   
    answer = "|".join(step2)
    #print(answer)
    
    qa2_4.update({q:answer})

qa2_4 = pd.DataFrame.from_dict(qa2_4, orient = "index")
qa2_4.reset_index(inplace=True)
qa2_4 = qa2_4.dropna()
qa2_4.columns = ["q","a"]

qa2_4.to_csv("qa//2_4.txt", sep='\t', index=False, header = None)
qa2_4.to_csv("qa//2_4.csv", index=False, header = None)
qa2_4.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)

### q2_5 (type) what are [category] companies in [coun]

In [290]:
q2_5 = "what are [category] companies in [coun]"

In [312]:
l2_51 = oc["country_code"].value_counts().rename_axis('country_code').reset_index(name='counts')
print (l2_51)

#sample2_51 =  l2_51["country_code"].to_list()[50:55]
sample2_51 = random.sample(l2_51["country_code"].to_list(), 10)
sample2_51.extend(["USA", "CHN", "CAN", "IND"])
sample2_51 = list(set(sample2_51))
print(sample2_51)


l2_52 = cg["category_list"].value_counts().rename_axis('category_list').reset_index(name='counts')
print (l2_52)

sample2_52 = ["Big_Data", "Virtual_Reality", 'Autonomous_Vehicles', 'Robotics', "Cloud_Data_Services", "EdTech", "CleanTech", "Computer_Vision", "Biofuel", "Air_Transportation", "Cloud_Storage", "Facial_Recognition", "Crowdourcing", 'Intellectual_Property', 'Intelligent_Systems',"Software_Engineering", 'Renewable_Energy', "Bioinformatics", "Clean_Energy", "Human_Computer_Interaction"]
print(sample2_52)

    country_code  counts
0            USA  644167
1            GBR  111975
2            IND   62839
3            DEU   55360
4            CAN   54831
..           ...     ...
211          COK       1
212          TKM       1
213          SJM       1
214          ERI       1
215          BVT       1

[216 rows x 2 columns]
['IND', 'NER', 'AZE', 'GLP', 'LBR', 'IMN', 'KOR', 'KAZ', 'AGO', 'CAN', 'CHN', 'REU', 'USA', 'NCL']
              category_list  counts
0                  Software  218600
1    Information_Technology  206780
2                Consulting  121849
3             Manufacturing  120584
4        Financial_Services  116207
..                      ...     ...
738                 Darknet       9
739                    Roku       7
740                    RISC       4
741        Ultimate_Frisbee       3
742            Generation_Y       1

[743 rows x 2 columns]
['Big_Data', 'Virtual_Reality', 'Autonomous_Vehicles', 'Robotics', 'Cloud_Data_Services', 'EdTech', 'CleanTech', 'Compute

In [313]:
#print(sorted(cg.category_list.unique()))

In [314]:
new3 = pd.merge(cg, oc, on = "name_id")
print(new3)

                  name_id relation_x            category_list    relation_y  \
0          Wetpaint@33abd   category               Publishing  country_code   
1          Wetpaint@33abd   category             Social_Media  country_code   
2          Wetpaint@33abd   category  Social_Media_Management  country_code   
3              Zoho@7efc7   category          Cloud_Computing  country_code   
4              Zoho@7efc7   category            Collaboration  country_code   
...                   ...        ...                      ...           ...   
3999848  LiveBeyond@f675c   category                  Charity  country_code   
3999849  LiveBeyond@f675c   category               Child_Care  country_code   
3999850  LiveBeyond@f675c   category                 Children  country_code   
3999851  LiveBeyond@f675c   category                Education  country_code   
3999852  LiveBeyond@f675c   category               Non_Profit  country_code   

        country_code  
0                USA  
1    

In [318]:
qa2_5 = dict()
for coun in sample2_51:
    for ca in sample2_52:
        #print(coun)
        q = q2_5.replace("[coun]", "[" + coun + "]")
        q = q.replace("[category]", "["+ca+"]")
        
        mask3 = (new3['country_code']==coun) & (new3['category_list']==ca)
        answerdf3 = new3[mask3]
        #print(answerdf)
        if len(answerdf3) > 0:
            step1 = answerdf3["name_id"].tolist()
                
            answer = "|".join(step1)
            
            #print(q)
            #print(answer)
            qa2_5.update({q:answer})
            
qa2_5 = pd.DataFrame.from_dict(qa2_5, orient = "index")
qa2_5.reset_index(inplace=True)
qa2_5 = qa2_5.dropna()
qa2_5.columns = ["q","a"]

qa2_5.to_csv("qa//2_5.txt", sep='\t', index=False, header = None)
qa2_5.to_csv("qa//2_5.csv", index=False, header = None)
qa2_5.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)

### q3_4 (explicit temporal + type) [gaming] acquisitions in [2021]

In [504]:
q3_4 = "[category] companies acquired in [year]"

In [505]:
ae1 = ae[["uuid", "ename_id"]]
ae1

,uuid,ename_id
0,ac@c62807f2-3487-f63b-e7fc-559b752ed44c,Flektor@330ba
1,ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca,GrandCentral@e1192
2,ac@01bd2058-ccf8-6102-f253-9224a67a592b,Last.fm@85f2d
3,ac@831becd6-e913-b9f5-ac09-2bff915f9453,Kosiso_-_store@a9f41
4,ac@9da90ee7-aad7-e0b4-8456-839363032e5e,Recipezaar@985a0
...,...,...
127181,ac@40f434aa-322b-477d-b7d8-543bc5a38367,Ultimedia@cdb8f
127182,ac@c6e14357-eeb8-4d65-b03f-4618753627ac,Digital_Guardian@7d676
127183,ac@f477824b-ea69-457a-9603-47552e002569,Brandwise@cf576
127184,ac@9811452d-9bfb-48ac-a5be-6b11f591ac70,Midwest_Warehouse_&_Distribution_System@60098


# 

In [506]:
ao = pd.read_csv("triples//acquired_on.csv")
ao = ao[["uuid", "acquired_on"]]
ao

,uuid,acquired_on
0,ac@c62807f2-3487-f63b-e7fc-559b752ed44c,2007
1,ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca,2007
2,ac@01bd2058-ccf8-6102-f253-9224a67a592b,2007
3,ac@831becd6-e913-b9f5-ac09-2bff915f9453,2007
4,ac@9da90ee7-aad7-e0b4-8456-839363032e5e,2007
...,...,...
127181,ac@40f434aa-322b-477d-b7d8-543bc5a38367,2021
127182,ac@c6e14357-eeb8-4d65-b03f-4618753627ac,2021
127183,ac@f477824b-ea69-457a-9603-47552e002569,2021
127184,ac@9811452d-9bfb-48ac-a5be-6b11f591ac70,2021


In [507]:
new5 = pd.merge(ae1, ao, on = "uuid")
new5.columns = ["uuid", "name_id", "acquired_on"]
new5

,uuid,name_id,acquired_on
0,ac@c62807f2-3487-f63b-e7fc-559b752ed44c,Flektor@330ba,2007
1,ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca,GrandCentral@e1192,2007
2,ac@01bd2058-ccf8-6102-f253-9224a67a592b,Last.fm@85f2d,2007
3,ac@831becd6-e913-b9f5-ac09-2bff915f9453,Kosiso_-_store@a9f41,2007
4,ac@9da90ee7-aad7-e0b4-8456-839363032e5e,Recipezaar@985a0,2007
...,...,...,...
127181,ac@40f434aa-322b-477d-b7d8-543bc5a38367,Ultimedia@cdb8f,2021
127182,ac@c6e14357-eeb8-4d65-b03f-4618753627ac,Digital_Guardian@7d676,2021
127183,ac@f477824b-ea69-457a-9603-47552e002569,Brandwise@cf576,2021
127184,ac@9811452d-9bfb-48ac-a5be-6b11f591ac70,Midwest_Warehouse_&_Distribution_System@60098,2021


In [508]:
cg

,name_id,relation,category_list
0,Wetpaint@33abd,category,Publishing
1,Wetpaint@33abd,category,Social_Media
2,Wetpaint@33abd,category,Social_Media_Management
3,Zoho@7efc7,category,Cloud_Computing
4,Zoho@7efc7,category,Collaboration
...,...,...,...
4304291,LiveBeyond@f675c,category,Charity
4304292,LiveBeyond@f675c,category,Child_Care
4304293,LiveBeyond@f675c,category,Children
4304294,LiveBeyond@f675c,category,Education


In [509]:
new51 = pd.merge(cg, new5, on = "name_id")

In [510]:
new51

,name_id,relation,category_list,uuid,acquired_on
0,Wetpaint@33abd,category,Publishing,ac@7172b9aa-74c5-e5a7-eb6e-9211ed79c605,2013
1,Wetpaint@33abd,category,Social_Media,ac@7172b9aa-74c5-e5a7-eb6e-9211ed79c605,2013
2,Wetpaint@33abd,category,Social_Media_Management,ac@7172b9aa-74c5-e5a7-eb6e-9211ed79c605,2013
3,Digg@553d0,category,Internet,ac@3b5487a3-1ec5-913f-bfe1-bc99ae3537ad,2012
4,Digg@553d0,category,Internet,ac@72024624-713a-4feb-8ea7-f72eae150f7b,2018
...,...,...,...,...,...
320451,Proximo_Resources@2b2e3,category,Natural_Resources,ac@13cf3dc3-6cc6-493e-88dc-86a2ceb9a24c,2021
320452,Ultimedia@cdb8f,category,Information_Services,ac@40f434aa-322b-477d-b7d8-543bc5a38367,2021
320453,Ultimedia@cdb8f,category,Software,ac@40f434aa-322b-477d-b7d8-543bc5a38367,2021
320454,Midwest_Warehouse_&_Distribution_System@60098,category,Logistics,ac@9811452d-9bfb-48ac-a5be-6b11f591ac70,2021


In [517]:
sample3_4 = ['Big_Data', "Natural_Resources", "Supply_Chain_Management", 'Virtual_Reality', 'Autonomous_Vehicles', 'Robotics', 'Cloud_Data_Services', 'EdTech', 'CleanTech', 'Computer_Vision', 'Biofuel', 'Air_Transportation', 'Cloud_Storage', 'Facial_Recognition', 'Crowdourcing', 'Intellectual_Property', 'Intelligent_Systems', 'Software_Engineering', 'Renewable_Energy', "Publishing", "Social_Media", "Information_Services	", "Logistics", "Internet", 'Bioinformatics', 'Clean_Energy', 'Human_Computer_Interaction']
sample3_4.extend(random.sample(list(set(new51.category_list.to_list())), 200))
sample3_4 = list(set(sample3_4))
print(sample3_4)

['EdTech', 'Smart_Building', 'Wellness', 'Intelligent_Systems', 'Management_Information_Systems', 'WebOS', 'Enterprise_Software', 'Lifestyle', 'Social_News', 'Prediction_Markets', 'Blogging_Platforms', 'Car_Sharing', 'Sales', 'Cooking', 'CMS', 'Virtual_Desktop', 'Therapeutics', 'Biotechnology', 'Private_Cloud', 'Direct_Marketing', 'Asset_Management', 'Lead_Generation', 'PaaS', 'Autonomous_Vehicles', 'Clinical_Trials', 'Consumer_Software', 'Continuing_Education', 'Content_Marketing', 'Social_Media', 'Real_Estate', 'Publishing', 'Photo_Sharing', 'Cloud_Storage', 'Wood_Processing', 'Air_Transportation', 'Green_Building', 'Aquaculture', 'Animal_Feed', 'Clean_Energy', 'Management_Consulting', 'Virtual_World', 'Supply_Chain_Management', 'Electrical_Distribution', 'Natural_Resources', 'Non_Profit', 'Enterprise', 'Handmade', 'Consumer_Electronics', 'Business_Travel', 'Mineral', 'Pet', 'Marine_Transportation', 'Robotics', 'Broadcasting', 'Parks', 'Private_Social_Networking', 'Consumer_Reviews',

In [518]:
years = [2020, 2021, 2000]

In [519]:
qa3_4 = dict()
for ca in sample3_4:
    for y in years:
        q = q3_4.replace("[category]", "[" + ca + "]")
        q = q.replace("[year]", "[" + str(y) + "]")
        mask5 = (new51["acquired_on"]==y) & (new51['category_list'] == ca)
        answer5 = new51[mask5].name_id.to_list()
        if len(answer5) > 0:
            answer = "|".join(answer5)
            #print(q)
            #print(answer)                         
            qa3_4.update({q:answer})
        
qa3_4 = pd.DataFrame.from_dict(qa3_4, orient = "index")
qa3_4.reset_index(inplace=True)
qa3_4 = qa3_4.dropna()
qa3_4.columns = ["q","a"]

qa3_4.to_csv("qa//3_4.txt", sep='\t', index=False, header = None)
qa3_4.to_csv("qa//3_4.csv", index=False, header = None)
qa3_4.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)

### q3_5 early stage investors in [org]

In [520]:
q3_5 = "early stage investors in [org]"

In [521]:
fro = pd.read_csv("triples//funding_round_launched_by.csv")
fro = fro[["uuid", "org_name_id"]]
fro

,uuid,org_name_id
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,Facebook@04fbb
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Facebook@04fbb
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Facebook@04fbb
3,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,Photobucket@018f9
4,fr@60e6afd9-1215-465a-dd17-0ed600d4e29b,Geni@b3098
...,...,...
438023,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Buffalo_Market@f6476
438024,fr@bd9228ee-ba5e-4a2d-b9ba-fb629a223ee9,Accern@40b4a
438025,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Let's_Enhance@15cd9
438026,fr@4ecac903-0feb-4b52-bfed-f8d9423c7e2d,TieSet@d7140


In [522]:
fr_inv = pd.read_csv("triples//investor_funding_round.csv")
fr_inv = fr_inv[["funding_round_uuid", "iname_id"]]
fr_inv.columns = ["uuid","iname_id"]
fr_inv

,uuid,iname_id
0,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Accel@7bf36
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Accel@7bf36
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Greylock@8a7e3
3,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Meritech_Capital_Partners@ab86f
4,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,Trinity_Ventures@275b9
...,...,...
130222,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Construct_Capital@12147
130223,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Precursor_Ventures@ebd60
130224,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Chamaeleon@60767
130225,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Mathias_Eklof@39480


In [523]:
fra = pd.read_csv("triples//funding_round_announced_on.csv")
fra = fra[["uuid", "announced_on"]]
fra["announced_on"] = fra["announced_on"].apply(lambda x: x[:4])
fra

,uuid,announced_on
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,2004
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,2005
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,2006
3,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,2006
4,fr@60e6afd9-1215-465a-dd17-0ed600d4e29b,2007
...,...,...
438025,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,2021
438026,fr@bd9228ee-ba5e-4a2d-b9ba-fb629a223ee9,2021
438027,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,2021
438028,fr@4ecac903-0feb-4b52-bfed-f8d9423c7e2d,2020


In [524]:
new4 = pd.merge(fro, fra, on = "uuid")
print(new4)
new4 = pd.merge(new4, fr_inv, on = "uuid")
new4

                                           uuid           org_name_id  \
0       fr@8a945939-18e0-cc9d-27b9-bf33817b2818        Facebook@04fbb   
1       fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb        Facebook@04fbb   
2       fr@6fae3958-a001-27c0-fb7e-666266aedd78        Facebook@04fbb   
3       fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846     Photobucket@018f9   
4       fr@60e6afd9-1215-465a-dd17-0ed600d4e29b            Geni@b3098   
...                                         ...                   ...   
438023  fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382  Buffalo_Market@f6476   
438024  fr@bd9228ee-ba5e-4a2d-b9ba-fb629a223ee9          Accern@40b4a   
438025  fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2   Let's_Enhance@15cd9   
438026  fr@4ecac903-0feb-4b52-bfed-f8d9423c7e2d          TieSet@d7140   
438027  fr@27c0cce5-b4db-4bb4-a19c-08c93585fa98          TieSet@d7140   

       announced_on  
0              2004  
1              2005  
2              2006  
3              2006  
4            

,uuid,org_name_id,announced_on,iname_id
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,Facebook@04fbb,2004,Western_Technology_Investment@cd866
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Facebook@04fbb,2005,Accel@7bf36
2,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Facebook@04fbb,2005,Accel@7bf36
3,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Facebook@04fbb,2005,Cambrian_Ventures@5f8bc
4,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Facebook@04fbb,2005,Cambrian_Ventures@5f8bc
...,...,...,...,...
130222,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Buffalo_Market@f6476,2021,Construct_Capital@12147
130223,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Buffalo_Market@f6476,2021,Precursor_Ventures@ebd60
130224,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Let's_Enhance@15cd9,2021,Chamaeleon@60767
130225,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Let's_Enhance@15cd9,2021,Mathias_Eklof@39480


In [525]:
l3_51 = new4["org_name_id"].value_counts().rename_axis('org_name_id').reset_index(name='counts')
print (l3_51)

                       org_name_id  counts
0                    Plastiq@301fd      68
1                        Box@5a9ee      54
2                   DocuSign@9ae6c      54
3                       Uber@bfcdc      54
4      EndoGastric_Solutions@e02d9      51
...                            ...     ...
46454    Incomparable_Things@bee9e       1
46455               Justsnap@49f29       1
46456        HighFive_Mobile@2192f       1
46457               FitCause@8ab52       1
46458    Home_Care_Delivered@4a4e0       1

[46459 rows x 2 columns]


In [526]:
print(l3_51.org_name_id.to_list()[1000:1050])

['Medium@35c41', 'Scale_AI@df74e', 'Wayspring@fe4ed', 'Cyphort@2ed0d', 'Outfittery@504e8', 'Benchling@79e4d', 'ManoMano@b4038', 'SEVEN_Networks@f0c0d', 'Jobvite@42b49', 'Urbint@60011', 'Entellus_Medical@4e6c1', 'C9_Inc.@5bad7', 'Avedro@06363', 'Vectra@c3194', 'LogicGate@0a888', 'Tulip_Retail@389df', 'OneWeb@03e65', 'Binx_Health@5fc13', 'enVista@c9723', 'NGINX@ce2cf', 'FarmLogs@e2061', 'Claroty@1de83', 'Tealium@b5841', 'Crate.io@57026', 'MadeiraMadeira@9c765', 'SambaNova_Systems@2d071', 'BioCatch@a591e', 'My1Login@3db9f', 'Naborly@c3444', 'Kik@f957b', 'Netlify@b2f7e', 'Visterra@4f574', 'Handle_Financial@5f114', 'Qwilt@cf843', 'Flatiron_Health@f6692', 'Flipboard@ad7f4', 'InsideView@efc1f', 'ActionIQ@2c058', 'Beamreach@ef24b', 'Everyday_Health_Group@184ac', 'DNAtriX@d2f02', 'Ordermark@a5e8a', '23andMe@77a41', 'Backupify@bdc54', 'Stella_Connect@e15ce', 'Jibe@ac8e1', 'Impossible_Foods@18b22', 'Veritas_Finance@12678', 'ExecOnline@6d818', 'Snap@d0dab']


In [527]:
sample3_51 = [ "Lenovo@7b7f1", "Microsoft@fd807", "Microsoft_Outlook@6a4e6",  "Facebook@df662", 'Uber@bfcdc',"Twitter@1f709", "Spotify@b1a5e", "Pinterest@8f83e", "Slack@6a285", "Deliveroo@a40d0", "Everyday_Health_Group@db417", "Tongdun_Technology@59eae", "Swire_Properties@840c7", "Air_France_KLM@aca18", "	Cathay_Pacific_Airways@a64f6", "Amazon@05554", "CVSPharmacy@f35cf", "AT&T_CyberSecurity@4ddb7", "FedEx@7c021", "The_Boeing_Company@4d760", "Syscor@45464","The_Pfizer_Foundation@117a4", "Starbucks_Corporation@68dd3", "Macy's@9fd1e", "Dollar_Tree@7d62e",  "	Netflix@3a7ec", "eBay@e56b0", "Bank_of_America@a7c69"]
sample3_51.extend(random.sample(l3_51.org_name_id.to_list(), 150))
sample3_51 = list(set(sample3_51))
print(len(sample3_51))

178


In [528]:
qa3_5 = dict()
for org in sample3_51:
    q = q3_5.replace("[org]", "[" + org + "]")
    step1 = new4.loc[new4["org_name_id"] == org, "uuid"].to_list()
    if len(step1) > 0:
        year = []
        for item in step1:
            year.extend(new4.loc[new4["uuid"] == item, "announced_on"].to_list())
        minyear = min(year)
        
        mask4 = (new4["org_name_id"]==org) & (new4['announced_on']==minyear)
        
        answer4 = new4[mask4].iname_id.to_list()
        
        answer = "|".join(answer4)
        qa3_5.update({q:answer})
        
qa3_5 = pd.DataFrame.from_dict(qa3_5, orient = "index")
qa3_5.reset_index(inplace=True)
qa3_5 = qa3_5.dropna()
qa3_5.columns = ["q","a"]

qa3_5.to_csv("qa//3_5.txt", sep='\t', index=False, header = None)
qa3_5.to_csv("qa//3_5.csv", index=False, header = None)
qa3_5.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)


### q4_1 (type) investments in [coun] [category] companies are done by who

In [602]:
q4_1 = "investments in [coun] [category] companies are done by who"

In [603]:
fro

,uuid,org_name_id
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,Facebook@04fbb
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Facebook@04fbb
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Facebook@04fbb
3,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,Photobucket@018f9
4,fr@60e6afd9-1215-465a-dd17-0ed600d4e29b,Geni@b3098
...,...,...
438023,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Buffalo_Market@f6476
438024,fr@bd9228ee-ba5e-4a2d-b9ba-fb629a223ee9,Accern@40b4a
438025,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Let's_Enhance@15cd9
438026,fr@4ecac903-0feb-4b52-bfed-f8d9423c7e2d,TieSet@d7140


In [604]:
fr_inv

,uuid,iname_id
0,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Accel@7bf36
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Accel@7bf36
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Greylock@8a7e3
3,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Meritech_Capital_Partners@ab86f
4,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,Trinity_Ventures@275b9
...,...,...
130222,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Construct_Capital@12147
130223,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Precursor_Ventures@ebd60
130224,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Chamaeleon@60767
130225,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Mathias_Eklof@39480


In [605]:
new61 = pd.merge(fr_inv, fro, on = "uuid").drop_duplicates()
new61.columns = ["uuid", "iname_id", "name_id"]
new61

,uuid,iname_id,name_id
0,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Accel@7bf36,Facebook@04fbb
2,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Cambrian_Ventures@5f8bc,Facebook@04fbb
4,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Greylock@8a7e3,Facebook@04fbb
5,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Meritech_Capital_Partners@ab86f,Facebook@04fbb
6,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Founders_Fund@7175c,Facebook@04fbb
...,...,...,...
130222,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Construct_Capital@12147,Buffalo_Market@f6476
130223,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Precursor_Ventures@ebd60,Buffalo_Market@f6476
130224,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Chamaeleon@60767,Let's_Enhance@15cd9
130225,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Mathias_Eklof@39480,Let's_Enhance@15cd9


In [606]:
new62 = pd.merge(new61, cg, on = "name_id").drop_duplicates()
new62 = pd.merge(new62, oc, on = "name_id").drop_duplicates()
new62

,uuid,iname_id,name_id,relation_x,category_list,relation_y,country_code
0,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Accel@7bf36,Facebook@04fbb,category,Mobile_Apps,country_code,USA
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Accel@7bf36,Facebook@04fbb,category,Photo_Sharing,country_code,USA
2,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Accel@7bf36,Facebook@04fbb,category,Social_Media,country_code,USA
3,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Accel@7bf36,Facebook@04fbb,category,Social_Network,country_code,USA
4,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Cambrian_Ventures@5f8bc,Facebook@04fbb,category,Mobile_Apps,country_code,USA
...,...,...,...,...,...,...,...
477699,fr@2853f8d1-eb98-425b-b476-a417aecba654,Flori_Ventures@36dbb,Coinprofile@7a1b7,category,Bitcoin,country_code,NGA
477700,fr@2853f8d1-eb98-425b-b476-a417aecba654,Flori_Ventures@36dbb,Coinprofile@7a1b7,category,Blockchain,country_code,NGA
477701,fr@2853f8d1-eb98-425b-b476-a417aecba654,Flori_Ventures@36dbb,Coinprofile@7a1b7,category,Cryptocurrency,country_code,NGA
477702,fr@2853f8d1-eb98-425b-b476-a417aecba654,Flori_Ventures@36dbb,Coinprofile@7a1b7,category,FinTech,country_code,NGA


In [607]:
print(sorted(new62.country_code.unique()))

['ALB', 'ARE', 'ARG', 'ARM', 'ASM', 'AUS', 'AUT', 'AZE', 'BAH', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BLR', 'BLZ', 'BMU', 'BRA', 'BRB', 'BWA', 'CAN', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COL', 'CRI', 'CYM', 'CYP', 'CZE', 'DEU', 'DNK', 'DZA', 'ECU', 'EGY', 'ESP', 'EST', 'ETH', 'FIN', 'FRA', 'FRO', 'GBR', 'GEO', 'GGY', 'GHA', 'GIB', 'GIN', 'GRC', 'GTM', 'HKG', 'HND', 'HRV', 'HUN', 'IDN', 'IMN', 'IND', 'IOT', 'IRL', 'IRQ', 'ISL', 'ISR', 'ITA', 'JEY', 'JOR', 'JPN', 'KAZ', 'KEN', 'KHM', 'KNA', 'KOR', 'KWT', 'LAO', 'LBN', 'LIE', 'LKA', 'LTU', 'LUX', 'LVA', 'MAF', 'MAR', 'MDA', 'MDG', 'MEX', 'MKD', 'MLI', 'MLT', 'MMR', 'MNG', 'MOZ', 'MUS', 'MWI', 'MYS', 'NGA', 'NLD', 'NOR', 'NPL', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'POL', 'PRI', 'PRT', 'PRY', 'PSE', 'ROM', 'RUS', 'RWA', 'SAU', 'SEN', 'SGP', 'SLE', 'SLV', 'SRB', 'SVK', 'SVN', 'SWE', 'SYC', 'TAN', 'TGO', 'THA', 'TJK', 'TTO', 'TUN', 'TUR', 'TWN', 'UGA', 'UKR', 'URY', 'USA', 'VEN', 'VNM', 'ZAF', 'ZMB', 'ZWE']


In [608]:
l4_1 = new62["country_code"].value_counts().rename_axis('country_code').reset_index(name='counts')
print (l4_1)


sample4_1 = ["USA", "CHN", "IND", "CAN", "RUS", "SGP", "AUS"]
sample4_1.extend(l4_1.country_code.to_list()[:15])
sample4_1.extend(random.sample(list(new62.country_code.unique()), 15))
sample4_1 = list(set(sample4_1))
print(sample4_1)


l4_2 = new62["category_list"].value_counts().rename_axis('category_list').reset_index(name='counts')
print (l4_2)


sample4_2 = ["Big_Data", "Virtual_Reality", 'Autonomous_Vehicles', 'Robotics', "Cloud_Data_Services", "EdTech", "CleanTech", "Computer_Vision", "Biofuel", "Air_Transportation", "Cloud_Storage", "Facial_Recognition", "Crowdourcing", 'Intellectual_Property', 'Intelligent_Systems',"Software_Engineering", 'Renewable_Energy', "Bioinformatics", "Clean_Energy", "Human_Computer_Interaction"]
sample4_2.extend(l4_2.category_list.to_list()[:15])
sample4_2.extend(random.sample(list(new62.category_list.unique()), 20))
sample4_2 = list(set(sample4_2))
print(sample4_2)

    country_code  counts
0            USA  320306
1            GBR   28046
2            IND   15697
3            CAN   14178
4            DEU   12796
..           ...     ...
135          NPL       2
136          BLZ       2
137          BFA       2
138          LAO       1
139          HND       1

[140 rows x 2 columns]
['IND', 'BRA', 'FRA', 'BEL', 'GBR', 'CHN', 'HKG', 'CHE', 'RUS', 'IMN', 'SWE', 'ESP', 'ISR', 'JPN', 'UKR', 'UGA', 'GGY', 'BAH', 'AUS', 'USA', 'BRB', 'COD', 'BLR', 'KOR', 'MEX', 'ALB', 'CAN', 'MDA', 'SGP', 'DEU']
              category_list  counts
0                  Software   30998
1               Health_Care   15939
2    Information_Technology   13996
3                      SaaS   13111
4             Biotechnology   11138
..                      ...     ...
721                   Parks       1
722                 Cricket       1
723                   Rugby       1
724       Ports_and_Harbors       1
725                  Diving       1

[726 rows x 2 columns]
['EdTech'

In [609]:
qa4_1 = dict()
for coun in sample4_1:
    print(coun)
    for ca in sample4_2:
        
        q = q4_1.replace("[coun]", "[" + coun + "]")
        q = q.replace("[category]", "[" + ca + "]")
        
        
        mask4_1 = (new62["category_list"] == ca) & (new62["country_code"] == coun)
        answer4_1 = new62[mask4_1].iname_id.to_list()
        
        if len(answer4_1) > 0:
            answer = "|".join(answer4_1)
            #print(q)
            #print(answer[:10])
            qa4_1.update({q:answer})
        
        '''
        step2 = []
        if len(answers4_1) >0:
            #print(answers4_1)
            for item in answers4_1:
                #print(item)
                #print(new61.loc[new61["org_name_id"] == item, "iname_id"])
                step2.extend(new61.loc[new61["org_name_id"] == item, "iname_id"].to_list())
                
            if len(step2) > 0:
                answer = "|".join(step2)
                print(q)
                print(answer)
                qa4_1.update({q:answer})
                '''

qa4_1 = pd.DataFrame.from_dict(qa4_1, orient = "index")
qa4_1.reset_index(inplace=True)
qa4_1 = qa4_1.dropna()
qa4_1.columns = ["q","a"]

qa4_1.to_csv("qa//4_1.txt", sep='\t', index=False, header = None)
qa4_1.to_csv("qa//4_1.csv", index=False, header = None)
qa4_1.to_csv("qa//qa.txt", sep='\t', mode = 'a', index=False, header = None)          

IND
BRA
FRA
BEL
GBR
CHN
HKG
CHE
RUS
IMN
SWE
ESP
ISR
JPN
UKR
UGA
GGY
BAH
AUS
USA
BRB
COD
BLR
KOR
MEX
ALB
CAN
MDA
SGP
DEU


### q2_6 (in/explicit temporal + type) Organizations that went public [this year]/ in [2021]

In [583]:
q2_6 = "Organizations that went public in [year]"

In [584]:
ip

,uuid,relation,went_public_on
0,ipo@72d30ebd-53ef-2486-6c29-22785c5173ce,went_public_on,1986
1,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4,went_public_on,1978
2,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b,went_public_on,2006
3,ipo@ee426509-826e-5dd0-9309-e79c8f384904,went_public_on,1999
4,ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e,went_public_on,1988
...,...,...,...
47110,ipo@38615b77-5553-4363-a0c0-6ddc1cb8c41a,went_public_on,2021
47111,ipo@729769bd-c70f-4b17-8b52-e9a0bb04a2a9,went_public_on,2021
47112,ipo@fe06296f-f088-48be-97d4-711b6f5ce494,went_public_on,2021
47113,ipo@374a7dd8-dd9d-4271-acb7-9ecd9c32417f,went_public_on,2021


In [585]:
oi

,name_id,relation,uuid
0,Microsoft@9ffbb,org_ipo,ipo@72d30ebd-53ef-2486-6c29-22785c5173ce
1,The_Walt_Disney_Company@f5695,org_ipo,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4
2,DivX@81ba9,org_ipo,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b
3,XO_Group@aa14b,org_ipo,ipo@ee426509-826e-5dd0-9309-e79c8f384904
4,BMC_Software@89245,org_ipo,ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e
...,...,...,...
47110,WeWork@7857c,org_ipo,ipo@38615b77-5553-4363-a0c0-6ddc1cb8c41a
47111,SAB_Biotherapeutics@b986e,org_ipo,ipo@729769bd-c70f-4b17-8b52-e9a0bb04a2a9
47112,Redbox@3b3f2,org_ipo,ipo@fe06296f-f088-48be-97d4-711b6f5ce494
47113,Informatica@ad2c9,org_ipo,ipo@374a7dd8-dd9d-4271-acb7-9ecd9c32417f


In [586]:
new7 = pd.merge(ip, oi, on = "uuid")
new7

,uuid,relation_x,went_public_on,name_id,relation_y
0,ipo@72d30ebd-53ef-2486-6c29-22785c5173ce,went_public_on,1986,Microsoft@9ffbb,org_ipo
1,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4,went_public_on,1978,The_Walt_Disney_Company@f5695,org_ipo
2,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b,went_public_on,2006,DivX@81ba9,org_ipo
3,ipo@ee426509-826e-5dd0-9309-e79c8f384904,went_public_on,1999,XO_Group@aa14b,org_ipo
4,ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e,went_public_on,1988,BMC_Software@89245,org_ipo
...,...,...,...,...,...
47110,ipo@38615b77-5553-4363-a0c0-6ddc1cb8c41a,went_public_on,2021,WeWork@7857c,org_ipo
47111,ipo@729769bd-c70f-4b17-8b52-e9a0bb04a2a9,went_public_on,2021,SAB_Biotherapeutics@b986e,org_ipo
47112,ipo@fe06296f-f088-48be-97d4-711b6f5ce494,went_public_on,2021,Redbox@3b3f2,org_ipo
47113,ipo@374a7dd8-dd9d-4271-acb7-9ecd9c32417f,went_public_on,2021,Informatica@ad2c9,org_ipo


In [587]:
years2 = [1990, 1999, 2000, 2008, 2019, 2020, 2021]

In [592]:
qa2_6 = dict()
for y in years2:
    q = q2_6.replace("[year]", "[" + str(y) + "]")
    answer2_6 = new7.loc[new7["went_public_on"] == str(y), "name_id"].to_list()
    answer = "|".join(answer2_6)
    qa2_6.update({q:answer})
    
qa2_6 = pd.DataFrame.from_dict(qa2_6, orient = "index")
qa2_6.reset_index(inplace=True)
qa2_6 = qa2_6.dropna()
qa2_6.columns = ["q","a"]

qa2_6.to_csv("qa//2_6.txt", sep='\t', index=False, header = None)
qa2_6.to_csv("qa//2_6.csv", index=False, header = None)

### q4_2 (explicit temporal)  [category] companies founded within the recent [year] years that received funding over [amount]

In [593]:
fro

,uuid,org_name_id
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,Facebook@04fbb
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Facebook@04fbb
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Facebook@04fbb
3,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,Photobucket@018f9
4,fr@60e6afd9-1215-465a-dd17-0ed600d4e29b,Geni@b3098
...,...,...
438023,fr@012ce68a-3ba7-4ac3-bd8d-1c5501022382,Buffalo_Market@f6476
438024,fr@bd9228ee-ba5e-4a2d-b9ba-fb629a223ee9,Accern@40b4a
438025,fr@5c1a88df-9be6-4980-aa1b-e3e0e7c881d2,Let's_Enhance@15cd9
438026,fr@4ecac903-0feb-4b52-bfed-f8d9423c7e2d,TieSet@d7140


In [595]:
cg

,name_id,relation,category_list
0,Wetpaint@33abd,category,Publishing
1,Wetpaint@33abd,category,Social_Media
2,Wetpaint@33abd,category,Social_Media_Management
3,Zoho@7efc7,category,Cloud_Computing
4,Zoho@7efc7,category,Collaboration
...,...,...,...
4304291,LiveBeyond@f675c,category,Charity
4304292,LiveBeyond@f675c,category,Child_Care
4304293,LiveBeyond@f675c,category,Children
4304294,LiveBeyond@f675c,category,Education


In [597]:
fo = pd.read_csv("triples//org_founded_on.csv")
fo

,name_id,relation,founded_on
0,Wetpaint@33abd,founded_on,2005
1,Zoho@7efc7,founded_on,1996
2,Digg@553d0,founded_on,2004
3,Omidyar_Network@a8eec,founded_on,2004
4,Facebook@04fbb,founded_on,2004
...,...,...,...
1285497,SoULuvMuziq@a121d,founded_on,2016
1285498,McGuire_Memorial@90fb3,founded_on,1963
1285499,Success_Chain@ea3ac,founded_on,2020
1285500,Capital_Estate@9a7b8,founded_on,2020


In [601]:
new8 = pd.merge(fo, cg, on = "name_id")
fro1 = fro[["uuid", "org_name_id"]]
fro1.columns = ["uuid", "name_id"]
new8 = pd.merge(new8, fro1, on = "name_id").drop_duplicates()
new8

,name_id,relation_x,founded_on,relation_y,category_list,uuid
0,Wetpaint@33abd,founded_on,2005,category,Publishing,fr@06c1218c-7e15-a01d-c277-42f07a2b3743
1,Wetpaint@33abd,founded_on,2005,category,Publishing,fr@9bb86e43-5e93-74d1-d52c-39e9cecfb648
2,Wetpaint@33abd,founded_on,2005,category,Publishing,fr@6e02dcbf-49e1-97f4-8ad7-ca6888a4982f
3,Wetpaint@33abd,founded_on,2005,category,Social_Media,fr@06c1218c-7e15-a01d-c277-42f07a2b3743
4,Wetpaint@33abd,founded_on,2005,category,Social_Media,fr@9bb86e43-5e93-74d1-d52c-39e9cecfb648
...,...,...,...,...,...,...
1557602,Sooper@2069f,founded_on,2021,category,E-Commerce,fr@2b4b65bd-257c-405d-8434-0191566ede99
1557603,Sooper@2069f,founded_on,2021,category,Financial_Services,fr@2b4b65bd-257c-405d-8434-0191566ede99
1557604,Sooper@2069f,founded_on,2021,category,Home_Improvement,fr@2b4b65bd-257c-405d-8434-0191566ede99
1557605,Sooper@2069f,founded_on,2021,category,Marketplace,fr@2b4b65bd-257c-405d-8434-0191566ede99


### q4_3 Which [coun] companies are acquired by [coun] companies

In [610]:
ae

,ename_id,relation,uuid
0,Flektor@330ba,is_acquiree,ac@c62807f2-3487-f63b-e7fc-559b752ed44c
1,GrandCentral@e1192,is_acquiree,ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca
2,Last.fm@85f2d,is_acquiree,ac@01bd2058-ccf8-6102-f253-9224a67a592b
3,Kosiso_-_store@a9f41,is_acquiree,ac@831becd6-e913-b9f5-ac09-2bff915f9453
4,Recipezaar@985a0,is_acquiree,ac@9da90ee7-aad7-e0b4-8456-839363032e5e
...,...,...,...
127181,Ultimedia@cdb8f,is_acquiree,ac@40f434aa-322b-477d-b7d8-543bc5a38367
127182,Digital_Guardian@7d676,is_acquiree,ac@c6e14357-eeb8-4d65-b03f-4618753627ac
127183,Brandwise@cf576,is_acquiree,ac@f477824b-ea69-457a-9603-47552e002569
127184,Midwest_Warehouse_&_Distribution_System@60098,is_acquiree,ac@9811452d-9bfb-48ac-a5be-6b11f591ac70


In [611]:
ar

,rname_id,relation,uuid
0,Fox_Interactive_Media@1b78b,is_acquirer,ac@c62807f2-3487-f63b-e7fc-559b752ed44c
1,Google@27711,is_acquirer,ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca
2,CBS_Entertainment@2e4ea,is_acquirer,ac@01bd2058-ccf8-6102-f253-9224a67a592b
3,Google@27711,is_acquirer,ac@831becd6-e913-b9f5-ac09-2bff915f9453
4,Scripps_Networks@a4b35,is_acquirer,ac@9da90ee7-aad7-e0b4-8456-839363032e5e
...,...,...,...
127181,Remarkable_Group_International@a9b35,is_acquirer,ac@40f434aa-322b-477d-b7d8-543bc5a38367
127182,HelpSystems@356cd,is_acquirer,ac@c6e14357-eeb8-4d65-b03f-4618753627ac
127183,MarketTime@98bf3,is_acquirer,ac@f477824b-ea69-457a-9603-47552e002569
127184,Ryder@93d6e,is_acquirer,ac@9811452d-9bfb-48ac-a5be-6b11f591ac70


In [612]:
oc

,name_id,relation,country_code
0,Wetpaint@33abd,country_code,USA
1,Zoho@7efc7,country_code,USA
2,Digg@553d0,country_code,USA
3,Omidyar_Network@a8eec,country_code,USA
4,Facebook@04fbb,country_code,USA
...,...,...,...
1456191,McGuire_Memorial@90fb3,country_code,USA
1456192,Success_Chain@ea3ac,country_code,GBR
1456193,Capital_Estate@9a7b8,country_code,MEX
1456194,FABAY@ff9d8,country_code,TUR


In [617]:
new8 = pd.merge(ae, ar, on = "uuid")

oce = oc[["name_id", "country_code"]]
oce.columns = ["ename_id", "country_code"]
new8 = pd.merge(new8, oce, on = "ename_id")
new8 = new8[["ename_id", "uuid", "rname_id", "country_code"]]
new8.columns = ["ename_id", "uuid", "rname_id", "ecountry_code"]

ocr  =oc[["name_id", "country_code"]]
ocr.columns = ["rname_id", "country_code"]
new8 = pd.merge(new8, ocr, on = "rname_id")
new8 = new8[["ename_id", "uuid", "rname_id", "ecountry_code", "country_code"]]
new8.columns = ["ename_id", "uuid", "rname_id", "ecountry_code", "rcountry_code"]

new8

,ename_id,uuid,rname_id,ecountry_code,rcountry_code
0,Flektor@330ba,ac@c62807f2-3487-f63b-e7fc-559b752ed44c,Fox_Interactive_Media@1b78b,USA,USA
1,IGN_Entertainment@aa15b,ac@cfe42458-5bff-f9e8-3fdf-21f037bd2726,Fox_Interactive_Media@1b78b,USA,USA
2,Newroo@4635c,ac@9a3b1e52-5951-5441-9a53-ff41224788a1,Fox_Interactive_Media@1b78b,USA,USA
3,Photobucket@018f9,ac@aefc3138-f42c-c692-01db-93dee9aa85b0,Fox_Interactive_Media@1b78b,USA,USA
4,Scout@37dd1,ac@50eaa826-e8c6-2392-c02c-d0f5e08c0413,Fox_Interactive_Media@1b78b,USA,USA
...,...,...,...,...,...
119799,Trishula@8c7fe,ac@a04678e6-9d13-4852-8db1-d40d36498d71,Trentar@18c21,IND,IND
119800,Dawson_Geophysical@58205,ac@39e87301-52e6-4bb3-9d30-70efcca644d4,Wilks_Brothers@bebca,USA,USA
119801,Proximo_Resources@2b2e3,ac@13cf3dc3-6cc6-493e-88dc-86a2ceb9a24c,Rugby_Mining_Limited@a6303,AUS,CAN
119802,Ultimedia@cdb8f,ac@40f434aa-322b-477d-b7d8-543bc5a38367,Remarkable_Group_International@a9b35,GBR,GBR


In [ ]:
sample4_2 = random.sample()